In [12]:
import pandas as pd



In [3]:
train_human=pd.read_csv(f"{path_to_multip_humandata}/multip_train.csv")
test_human=pd.read_csv(f"{path_to_multip_humandata}/multip_test.csv")
val_human=pd.read_csv(f"{path_to_multip_humandata}/multip_val.csv")

In [14]:
train_llm=pd.read_csv(f"{path_to_multip_llm_data}/multip_new/train_llm.csv")
val_llm=pd.read_csv(f"{path_to_multip_llm_data}/multip_new/test_llm.csv")
test_llm=pd.read_csv(f"{path_to_multip_llm_data}/multip_new/val_llm.csv")  

In [4]:
count_no_majority_train = train_human['majority_label'].value_counts().get("No majority", 0)

In [17]:
train_llm['llm_labels'] = train_llm.apply(lambda row: [row['olmo'], row['llama'], row['mistral']], axis=1)
val_llm['llm_labels'] = val_llm.apply(lambda row: [row['olmo'], row['llama'], row['mistral']], axis=1)
test_llm['llm_labels'] = test_llm.apply(lambda row: [row['olmo'], row['llama'], row['mistral']], axis=1)

In [28]:
labels = ['Pro', 'Against', 'Neutral', 'Not-about']
num_labels = len(labels)
id2label = {id:label for id,label in enumerate(labels)}
label2id = {label:id for id,label in enumerate(labels)}

In [29]:
import ast
def ensure_list(val):
    if isinstance(val, str):
        try:
            return ast.literal_eval(val)
        except (ValueError, SyntaxError):
            return [val]
    elif isinstance(val, list):
        return val
    else:
        return []

In [ ]:

train_human['labels'] = train_human['labels'].apply(ensure_list)
test_human['labels'] = test_human['labels'].apply(ensure_list)
val_human['labels'] = val_human['labels'].apply(ensure_list)


train_human['label_indices'] = train_human['labels'].apply(lambda x: [label2id[label] for label in x if label in label2id])
test_human['label_indices'] = test_human['labels'].apply(lambda x: [label2id[label] for label in x if label in label2id])
val_human['label_indices'] = val_human['labels'].apply(lambda x: [label2id[label] for label in x if label in label2id])

print(train_human[['labels', 'label_indices']].head())

In [20]:
train_llm['labels'] = train_llm['labels'].apply(ensure_list)
test_llm['labels'] = test_llm['labels'].apply(ensure_list)
val_llm['labels'] = val_llm['labels'].apply(ensure_list)


train_llm['label_indices'] = train_llm['labels'].apply(lambda x: [label2id[label] for label in x if label in label2id])
test_llm['label_indices'] = test_llm['labels'].apply(lambda x: [label2id[label] for label in x if label in label2id])
val_llm['label_indices'] = val_llm['labels'].apply(lambda x: [label2id[label] for label in x if label in label2id])

In [8]:
unique_labels = set(label for sublist in train_human['labels'] for label in sublist)
print("Etichette uniche nel DataFrame:", unique_labels)
print(train_human['labels'].head())

Etichette uniche nel DataFrame: {"['Neutral', 'Not-about', nan]", "['Against', 'Against', nan]", 'Against', 'Neutral', 'Link-broken', 'Not-about', 'Pro'}
0                      [Pro, Pro, Pro]
1          [Neutral, Neutral, Neutral]
2    [Not-about, Not-about, Not-about]
3              [Neutral, Neutral, Pro]
4          [Neutral, Neutral, Neutral]
Name: labels, dtype: object


In [21]:
unique_labels = set(label for sublist in train_llm['labels'] for label in sublist)
print("Etichette uniche nel DataFrame:", unique_labels)
print(train_llm['labels'].head())

Etichette uniche nel DataFrame: {'Neutral', 'Not-about', 'Pro', 'Against'}
0                      [Pro, Pro, Pro]
1          [Neutral, Neutral, Neutral]
2    [Not-about, Not-about, Not-about]
3              [Neutral, Neutral, Pro]
4          [Neutral, Neutral, Neutral]
Name: labels, dtype: object


In [32]:
from scipy.special import softmax


def create_soft_labels_index(data, label_column, col_name, all_labels=None):
    """
    Creates soft labels based on a "labels" column that contains all labels for a document,
    approximating them to one decimal point, ensuring that all classes have an assigned probability.

    :param data: pandas DataFrame containing the data
    :param label_column: Name of the column that contains the labels (default: 'labels')
    :param col_name: The name of the column to store the resulting soft labels
    :param all_labels: List of all possible labels. If None, they will be inferred from the labels in the DataFrame.
    :return: DataFrame with a new column containing the soft labels.
    """
    _data = data.copy()  # Create a copy of the input data to avoid modifying the original DataFrame

    # If all_labels is not provided, use the default label set
    if all_labels is None:
        all_labels = ['Pro', 'Against', 'Neutral', 'Not-about']

    all_labels_set = set(all_labels)  # Convert all_labels to a set for quick lookup (optional)

    soft_labels_list = []  # Initialize a list to store soft labels for each document

    # Iterate through each label in the specified label_column
    for labels in _data[label_column]:
        
        if isinstance(labels, str):
            labels = ast.literal_eval(labels)  # If labels are a string, convert them into a list using literal_eval

        # Count the occurrences of each label in the current list of labels
        label_counts = pd.Series(labels).value_counts().to_dict()

        # Create a list of counts, ensuring each label from all_labels is represented (default count = 0)
        counts = [label_counts.get(label, 0) for label in all_labels]

        # Compute soft probabilities using the softmax function
        soft_probs = softmax(counts)

        # Map each label to its corresponding softmax probability
        soft_label_map = dict(zip(all_labels, soft_probs))

        # Retrieve the soft probabilities for all labels in the same order as all_labels
        soft_labels = [soft_label_map[label] for label in all_labels]

        # Round each soft probability to one decimal point
        soft_labels = [round(prob, 1) for prob in soft_labels]

        # Append the soft labels to the list
        soft_labels_list.append(soft_labels)

    # Add a new column to the DataFrame with the generated soft labels
    _data[col_name] = soft_labels_list

    return _data  # Return the modified DataFrame

In [10]:
train_soft_human=create_soft_labels_index(train_human)
test_soft_human=create_soft_labels_index(test_human)
val_soft_human=create_soft_labels_index(val_human)

In [23]:
train_soft_llm=create_soft_labels_index(train_llm)
test_soft_llm=create_soft_labels_index(test_llm)
val_soft_llm=create_soft_labels_index(val_llm)

In [11]:
import os
directory = 'your dataset saving dir'
train_file = os.path.join(directory, 'train_soft_human.csv')
test_file = os.path.join(directory, 'test_soft_human.csv')
val_file = os.path.join(directory, 'val_soft_human.csv')


train_soft_human.to_csv(train_file, index=False)
test_soft_human.to_csv(test_file, index=False)
val_soft_human.to_csv(val_file, index=False) 




In [24]:
import os
directory = 'your dataset saving dir'
train_file = os.path.join(directory, 'train_soft_llm.csv')
test_file = os.path.join(directory, 'test_soft_llm.csv')
val_file = os.path.join(directory, 'val_soft_llm.csv')


train_soft_llm.to_csv(train_file, index=False)
test_soft_llm.to_csv(test_file, index=False)
val_soft_llm.to_csv(val_file, index=False) 